In [1]:
!python3 --version

Python 3.10.2


In [4]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgradev -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 135.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 KB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 KB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 28.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 KB 29.9 MB/s eta 0:00:00
  Using cached safetensors-0.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 117.1 MB/s eta 0:00:0000:0100:01
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 KB 68.5 MB/s eta 0:00:00
  Using cached reg

#### fix for could not import -lmza
https://github.com/lucidrains/imagen-pytorch/issues/92 

#### Fix for Can't import datasets AttributeError: partially initialized module 'datasets' has no attribute 'utils' (most likely due to a circular import)

Restart juypyter and kernel


In [2]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")

# dataset size: 15011


/home/ec2-user/environment/ai-project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset json (/home/ec2-user/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


dataset size: 15011


In [3]:
sample_ds = dataset[randrange(len(dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
Is plastic bad for us?
--> context

--> response
Have you ever thought about where your plastic water bottles go? It probably ends up in a landfill, where it will decompose for over 1000 years as it deposits toxic chemicals. Plastic is an invention that has done more harm than good. There are about 363,762,732,605 pounds of plastic covering the earth. Most of this plastic can’t be recycled. Although people think it is too expensive or inconvenient to get rid of plastics, we should reduce the amount of plastic used or increase the price of plastics. One reason is that when plastic invades a balanced ecosystem, it corrupts it completely. The marine biome is one that is affected by this product. The animals in the ocean die of suffocation, ingestion of plastics, and entanglement. The animals affected are seagulls, whales, seals, fish, and turtles, and that's just a few. The animals eat so much plastic that they die of constipation from it. Another reason is the presence of

In [4]:
set(dataset['category'])

{'brainstorming',
 'classification',
 'closed_qa',
 'creative_writing',
 'general_qa',
 'information_extraction',
 'open_qa',
 'summarization'}

In [5]:
# len(dataset.map( ['context']))

In [6]:
# list(filter(lambda x :x != '', list(map(lambda x:((x['category'], len(x['context'])) if len(x['context']) > 5000 else ''), dataset))))

To instruct tune our model, we need to convert our structured examples into a collection of tasks described via instructions. We define a formatting_function that takes a sample and returns a string with our format instruction.

In [7]:
def format_instruction(sample):
	return f"""### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.
            
            ### Input:
            {sample['response']}
            
            ### Response:
            {sample['instruction']}
        """


Let's test our formatting function on a random example.

In [8]:
from random import randrange

print(format_instruction(dataset[randrange(len(dataset))]))



### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.
            
            ### Input:
            The most important concept around climate change is that "weather", such as snowstorms and blizzards, is different then "climate". With climate change, the extremes are what fluctuate most: the colds are colder, and the hots are hotter. This sounds like your experience with blizzards and snowstorms. You may actually be experiencing more extreme cold weather as the planet warms due to greenhouse gases because of how the jet-stream is changing and warmer ocean temperatures put more water vapor into the atmosphere. Although it may seem that the weather is still cold at a local level, nearly every scientist agrees that climate change is causing more dramatic and extreme weather all around the world.
            
            ### Response:
            How can the climate be warming if I keep getting blizzards

In [9]:
print(format_instruction(sample_ds))

### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.
            
            ### Input:
            Have you ever thought about where your plastic water bottles go? It probably ends up in a landfill, where it will decompose for over 1000 years as it deposits toxic chemicals. Plastic is an invention that has done more harm than good. There are about 363,762,732,605 pounds of plastic covering the earth. Most of this plastic can’t be recycled. Although people think it is too expensive or inconvenient to get rid of plastics, we should reduce the amount of plastic used or increase the price of plastics. One reason is that when plastic invades a balanced ecosystem, it corrupts it completely. The marine biome is one that is affected by this product. The animals in the ocean die of suffocation, ingestion of plastics, and entanglement. The animals affected are seagulls, whales, seals, fish, and turtles, and th

### Instruction-tune Llama 2 using trl and the SFTTrainer

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, BitsAndBytesConfig

use_flash_attention = False
# COMMENT IN TO USE FLASH ATTENTION
# replace attention with flash attention
# if torch.cuda.get_device_capability()[0] >= 8:
#     from utils.llama_patch import replace_attn_with_flash_attn
#     print("Using flash attention")
#     replace_attn_with_flash_attn()
#     use_flash_attention = True




In [17]:
# Hugging Face model id
# model_id = "NousResearch/Llama-2-7b-hf" # non-gated
model_id = "google/flan-t5-large"
# model_id = "meta-llama/Llama-2-7b-hf" # gated


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="auto")
model.config.pretraining_tp = 1




















































































































Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-large and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



In [18]:



# Validate that the model is using flash attention, by comparing doc strings
if use_flash_attention:
    from utils.llama_patch import forward
    assert model.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, "Model is not using flash attention"


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

The SFTTrainer supports a native integration with peft, which makes it super easy to efficiently instruction tune LLMs. We only need to create our LoRAConfig and provide it to the trainer.

In [19]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


Before we can start our training we need to define the hyperparameters (TrainingArguments) we want to use.

In [20]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="flan-7-int4-dolly",
    num_train_epochs=3,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)


In [21]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)


In [23]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()


OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 21.99 GiB total capacity; 10.84 GiB already allocated; 659.75 MiB free; 11.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Test Model and run Inference

In [ ]:
if use_flash_attention:
    # unpatch flash attention
    from utils.llama_patch import unplace_flash_attn_with_attn
    unplace_flash_attn_with_attn()

import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

args.output_dir = "llama-7-int4-dolly"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)


Let’s load the dataset again with a random sample to try to generate an instruction.

In [ ]:
from datasets import load_dataset
from random import randrange


# Load dataset from the hub and get a sample
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
sample = dataset[randrange(len(dataset))]

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample['response']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample['instruction']}")
